<a href="https://colab.research.google.com/github/YounSooKimTech/Joy_Profile_Photos/blob/main/IVY_Yale_Meta_and_Smile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meta Dataset

In [3]:
df_department =[]
df_name =[]
df_title =[]
df_link =[]
df_url = []

import pandas as pd
import requests
from bs4 import BeautifulSoup

url = "https://som.yale.edu/faculty-research/faculty-directory"
response = requests.get(url)
#print(url, response)
soup = BeautifulSoup(response.content, 'html.parser')

div_tags = soup.find_all("div", class_="faculty-teaser-list-view__group")

for div_tag in div_tags:
  a_tags = div_tag.find_all("a")
  for a_tag in a_tags:
    name = a_tag.get_text().strip()
    href = a_tag["href"]
    url = f"https://som.yale.edu{href}"
    df_name.append(name)
    df_url.append(url)
    #print(name, ":", url)

df_meta = pd.DataFrame({"Name":df_name,
                     "URL":df_url})

df_meta.head()

,Name,URL
0,Jason Abaluck,https://som.yale.edu/faculty-research/faculty-...
1,Laura Adler,https://som.yale.edu/faculty-research/faculty-...
2,Rick Antle,https://som.yale.edu/faculty-research/faculty-...
3,Jaime Arellano-Bover,https://som.yale.edu/faculty-research/faculty-...
4,David Argente,https://som.yale.edu/faculty-research/faculty-...


In [4]:
import requests
from bs4 import BeautifulSoup

df_title = []

for index, row in df_meta.iterrows():
    name = row["Name"]
    url = row["URL"]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    p_tag = soup.find("p", class_ = "node__job-title")
    title = p_tag.get_text().strip()

    df_title.append(title)

    #print(name, ":", title)

df_meta["Title"] = df_title

df_meta.head(3)

,Name,URL,Title
0,Jason Abaluck,https://som.yale.edu/faculty-research/faculty-...,Professor of Economics
1,Laura Adler,https://som.yale.edu/faculty-research/faculty-...,Assistant Professor of Organizational Behavior
2,Rick Antle,https://som.yale.edu/faculty-research/faculty-...,William S. Beinecke Professor of Accounting


## Filter the data based on titles


In [5]:
df_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Name    236 non-null    object
 1   URL     236 non-null    object
 2   Title   236 non-null    object
dtypes: object(3)
memory usage: 5.7+ KB


In [6]:
df_SOM = df_meta.head(105) # <- Exclude Affiliated Faculty, Visiting Instructors & Scholars, Emeriti

df_SOM_prof = df_SOM[~df_SOM["Title"].str.contains("Lecturer|Visiting|Practice")]

df_SOM_prof.head(3)

,Name,URL,Title
0,Jason Abaluck,https://som.yale.edu/faculty-research/faculty-...,Professor of Economics
1,Laura Adler,https://som.yale.edu/faculty-research/faculty-...,Assistant Professor of Organizational Behavior
2,Rick Antle,https://som.yale.edu/faculty-research/faculty-...,William S. Beinecke Professor of Accounting


## Rank and Department

In [7]:
import numpy as np

conditions = [
    df_SOM_prof["Title"].str.contains("Emeritus"),
    df_SOM_prof["Title"].str.contains("Emerita"),
    df_SOM_prof["Title"].str.contains("Assistant"),
    df_SOM_prof["Title"].str.contains("Associate")
]

choices = ["Emeritus", "Emeritus", "Assistant", "Associate"]

df_SOM_prof["Rank"] = np.select(conditions, choices, default="Full")

df_SOM_prof["Rank"].value_counts()

<ipython-input-7-b0d3f1f0795b>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SOM_prof["Rank"] = np.select(conditions, choices, default="Full")


Full         46
Assistant    29
Associate    12
Name: Rank, dtype: int64

In [8]:
# Department

df_SOM_prof["Department"] = df_SOM_prof["Title"].str.split("of").str[-1]

df_SOM_prof.loc[df_SOM_prof["Name"] =="Lorenzo Caliendo", "Department"] = "Economics"
df_SOM_prof.loc[df_SOM_prof["Name"] =="Kerwin K. Charles", "Department"] = "Economics"

df_SOM_prof.loc[df_SOM_prof["Name"] =="Ravi Dhar", "Department"] = "Management and Marketing"
df_SOM_prof.loc[df_SOM_prof["Name"] =="William N. Goetzmann", "Department"] = "Finance and Management"

df_SOM_prof.loc[df_SOM_prof["Name"] =="Robert T. Jensen", "Department"] = "Economics"
df_SOM_prof.loc[df_SOM_prof["Name"] =="Bryan T. Kelly", "Department"] = "Finance"

df_SOM_prof.loc[df_SOM_prof["Name"] =="K. Geert Rouwenhorst", "Department"] = "Finance"
df_SOM_prof.loc[df_SOM_prof["Name"] =="K. Sudhir", "Department"] = "Marketing"

df_SOM_prof["Department"] = df_SOM_prof["Department"].str.replace("&", "and")
df_SOM_prof["Department"] = df_SOM_prof["Department"].str.strip()

<ipython-input-8-8da8df86873f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SOM_prof["Department"] = df_SOM_prof["Title"].str.split("of").str[-1]
<ipython-input-8-8da8df86873f>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SOM_prof["Department"] = df_SOM_prof["Department"].str.replace("&", "and")
<ipython-input-8-8da8df86873f>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [9]:
df_SOM_prof["Department"].value_counts()

Finance                     18
Economics                   14
Marketing                   14
Organizational Behavior      9
Accounting                   7
Operations Management        6
Economics and Management     4
Management                   3
Finance and Management       2
Management and Marketing     2
Management and Finance       1
Public Health                1
Finance and Economics        1
Operations                   1
Management and Economics     1
Operations Research          1
International Economics      1
Accounting and Finance       1
Name: Department, dtype: int64

## Sex Information based on Biography

In [10]:
import requests
from bs4 import BeautifulSoup

df_bio = []

for index, row in df_SOM_prof.iterrows():
    name = row["Name"]
    url = row["URL"]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    div_tag = soup.find("div", class_="text-long")
    bio_text = div_tag.get_text().strip()

    df_bio.append(bio_text)

    #print(bio_text, "\n")

df_SOM_prof["Biography"] = df_bio

<ipython-input-10-9cdee8077586>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SOM_prof["Biography"] = df_bio


In [11]:
import spacy
nlp = spacy.load('en_core_web_sm')

pronouns_list = []

for text in df_SOM_prof['Biography']:
    if text == None:
      text = ""
    doc = nlp(text)
    pronouns = [token.text.lower() for token in doc if token.pos_ == 'PRON' and token.text.lower() in ['he', 'she', 'her', 'his']]
    pronouns_list.append(pronouns)

df_SOM_prof['Pronouns'] = pronouns_list

def determine_sex(pronouns):
    if 'he' in pronouns or 'his' in pronouns:
        return 'Male'
    elif 'she' in pronouns or 'her' in pronouns:
        return 'Female'
    else:
        return 'Unknown'

df_SOM_prof['Sex'] = df_SOM_prof['Pronouns'].apply(determine_sex)

<ipython-input-11-4e6dfda9fd87>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SOM_prof['Pronouns'] = pronouns_list
<ipython-input-11-4e6dfda9fd87>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SOM_prof['Sex'] = df_SOM_prof['Pronouns'].apply(determine_sex)


### No Biography information case handlings

In [12]:
df_SOM_prof[df_SOM_prof["Sex"] == "Unknown"]

,Name,URL,Title,Rank,Department,Biography,Pronouns,Sex
23,Jason Dana,https://som.yale.edu/faculty-research/faculty-...,Associate Professor of Management and Marketing,Associate,Management and Marketing,Jason Dana’s research focuses on how people ma...,[],Unknown
37,Soheil Ghili,https://som.yale.edu/faculty-research/faculty-...,Assistant Professor of Marketing,Assistant,Marketing,I am an assistant professor of Marketing at Ya...,[],Unknown
52,Joowon Klusowski,https://som.yale.edu/faculty-research/faculty-...,Assistant Professor of Marketing,Assistant,Marketing,Professor Joowon Klusowski studies how people ...,[],Unknown
74,Nils Rudi,https://som.yale.edu/faculty-research/faculty-...,Professor of Operations Management,Full,Operations Management,"Education\nPhD, The Wharton School, University...",[],Unknown


In [13]:
# https://cbey.yale.edu/our-community/jason-dana
# https://fds.yale.edu/soheil-ghili/
# https://law.yale.edu/nils-rudi

df_SOM_prof.loc[df_SOM_prof["Name"] == "Jason Dana", "Sex"] = "Male"
df_SOM_prof.loc[df_SOM_prof["Name"] == "Soheil Ghili", "Sex"] = "Male"
df_SOM_prof.loc[df_SOM_prof["Name"] == "Nils Rudi", "Sex"] = "Male"


<ipython-input-13-dc406dd42505>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SOM_prof.loc[df_SOM_prof["Name"] == "Jason Dana", "Sex"] = "Male"
<ipython-input-13-dc406dd42505>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SOM_prof.loc[df_SOM_prof["Name"] == "Soheil Ghili", "Sex"] = "Male"
<ipython-input-13-dc406dd42505>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SOM_prof.loc[df_SOM_prof["Name"] == "Nils Rudi", "Sex

# Photo scrap

In [14]:
from google.colab import files
import requests
from bs4 import BeautifulSoup

for index, row in df_SOM_prof.iterrows():
    name = row["Name"]
    new_name = name.lower().replace(".", "").replace(" ", "_")
    filename = f"{new_name}.jpg"

    url = row["URL"]
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    div_tag = soup.find("div", class_="node__header-image")
    img_tag = div_tag.find("img")
    img_srcset = img_tag.get("srcset", None)

    if img_srcset is None:
        img_srcset = img_tag.get("data-src", None)
        img_src = img_srcset

    if img_srcset:
        # Split the srcset by space and get the first element
        img_src = img_srcset.split(",")[-1].strip()
        img_src = img_src.replace("1021w", "").strip()

        if "160" in img_src:
          img_src = img_tag.get("data-src")
          img_src = img_src.replace("square_160", "card_800_")


    img_url = f"https://som.yale.edu{img_src}"

    #print(name, img_url, "\n")
    flag = requests.get(img_url)

    with open(filename, "wb") as f:
        noop = f.write(flag.content)
        print(f"{name} saved the {filename}")

        #files.download(filename)
        #print(f"download the {filename}")

Jason Abaluck saved the jason_abaluck.jpg
Laura Adler saved the laura_adler.jpg
Rick Antle saved the rick_antle.jpg
David Argente saved the david_argente.jpg
Nicholas C. Barberis saved the nicholas_c_barberis.jpg
James N. Baron saved the james_n_baron.jpg
Barbara Biasi saved the barbara_biasi.jpg
Tristan L. Botelho saved the tristan_l_botelho.jpg
Alex Burnap saved the alex_burnap.jpg
Lorenzo Caliendo saved the lorenzo_caliendo.jpg
Kerwin K. Charles saved the kerwin_k_charles.jpg
Judith A. Chevalier saved the judith_a_chevalier.jpg
James Choi saved the james_choi.jpg
Christopher Clayton saved the christopher_clayton.jpg
Corey Cusimano saved the corey_cusimano.jpg
Jason Dana saved the jason_dana.jpg
Jennifer Dannals saved the jennifer_dannals.jpg
Ravi Dhar saved the ravi_dhar.jpg
Julia DiBenigno saved the julia_dibenigno.jpg
Kevin Donovan saved the kevin_donovan.jpg
Raphael Duguay saved the raphael_duguay.jpg
Jonathan S. Feinstein saved the jonathan_s_feinstein.jpg
Paul Fontanier saved t

In [15]:
df_SOM_prof.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87 entries, 0 to 104
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Name        87 non-null     object
 1   URL         87 non-null     object
 2   Title       87 non-null     object
 3   Rank        87 non-null     object
 4   Department  87 non-null     object
 5   Biography   87 non-null     object
 6   Pronouns    87 non-null     object
 7   Sex         87 non-null     object
dtypes: object(8)
memory usage: 6.1+ KB


# Smile Detection

In [16]:
# faces = face_cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=4)
# smiles = smile_cascade.detectMultiScale(roi_bottom_half, scaleFactor=1.5, minNeighbors=40)

import cv2
from google.colab.patches import cv2_imshow

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

names = []
face_detected = []
smile_detected = []

for index, row in df_SOM_prof.iterrows():
    name = row["Name"]
    new_name = name.lower().replace(".", "").replace(" ", "_")
    filename = f"{new_name}.jpg"

    img = cv2.imread(f'/content/{filename}', -1)

    faces = face_cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=4)

    if len(faces) > 0:
        face_var = 1
    else:
        face_var = 0

    for (x, y, w, h) in faces:
        # Draw rectangle around face
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        roi_bottom_half = img[y + int(h * 0.5):y + h, x:x + w]

        # Check if the region of interest is not None
        if roi_bottom_half is not None:
            smiles = smile_cascade.detectMultiScale(roi_bottom_half, scaleFactor=1.5, minNeighbors=40)

            if len(smiles) > 0:
                smile_var = 1
            else:
                smile_var = 0

            smile_coordinates = [(x + sx, y + int(h * 0.5) + sy, sw, sh) for (sx, sy, sw, sh) in smiles]
            # Calculate the bounding box for all smiles
            if smile_coordinates:
                min_x = min([x for (x, y, w, h) in smile_coordinates])
                min_y = min([y for (x, y, w, h) in smile_coordinates])
                max_w = max([x + w for (x, y, w, h) in smile_coordinates]) - min_x
                max_h = max([y + h for (x, y, w, h) in smile_coordinates]) - min_y

                # Draw a single larger rectangle around all smiles
                cv2.rectangle(img, (min_x, min_y), (min_x + max_w, min_y + max_h), (0, 255, 0), 1)

    face_detected.append(face_var)
    smile_detected.append(smile_var)
    names.append(name)


    #print(name)
    #cv2_imshow(img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
# Zeqiong Huang; Matthew Spiegel; Kosuke Uetake


In [17]:
df_smile = pd.DataFrame({"Name":names,
                         "Face":face_detected,
                         "Smile":smile_detected
                         })

# error handling

df_smile.loc[df_smile["Name"] == "Zeqiong Huang", "Face"] = 0
df_smile.loc[df_smile["Name"] == "Matthew Spiegel", "Face"] = 0
df_smile.loc[df_smile["Name"] == "Kosuke Uetake", "Face"] = 0

In [18]:
df_smile.head()

,Name,Face,Smile
0,Jason Abaluck,1,0
1,Laura Adler,1,1
2,Rick Antle,1,1
3,David Argente,1,1
4,Nicholas C. Barberis,1,1


# Save the File

In [19]:
df_result = df_SOM_prof.merge(df_smile, on="Name")

df_result["University"] = "Yale"

df_result = df_result[["University","Name","Department","Rank","Sex","Title","Pronouns","URL","Face","Smile"]]

In [20]:
#df_result.to_csv("Yale_meta_smile.csv", index=False)

In [21]:
df_result["Face"].value_counts()

1    74
0    13
Name: Face, dtype: int64

In [22]:
df_result["Smile"].value_counts()

1    53
0    34
Name: Smile, dtype: int64

In [23]:
df = df_result

df = df[df["Face"] == 1]

df = df[df["Sex"] != "Unknown"]

df = df[df["Rank"] != "Emeritus"]

from scipy.stats import chi2_contingency

contingency_table = pd.crosstab(df['Rank'], df['Smile'])
print(contingency_table, "\n")

chi2, p, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-square statistic: {chi2}")
print(f"P-value: {p}")
print(f"Degrees of freedom: {dof}")
print("Expected frequencies:")
print(expected)

Smile       0   1
Rank             
Assistant   8  18
Associate   4   5
Full       15  23 

Chi-square statistic: 0.7468645032209742
P-value: 0.6883676205143108
Degrees of freedom: 2
Expected frequencies:
[[ 9.61643836 16.38356164]
 [ 3.32876712  5.67123288]
 [14.05479452 23.94520548]]
